In [25]:
import folium
import fiona
import geopandas as gpd
from folium.plugins import MarkerCluster

In [26]:
# Create a map instance
m = folium.Map(
    location=[60.25, 24.8],
    tiles='Stamen Toner',
    zoom_start=9,
    control_scale=True,
    prefer_canvas=True #Optional
)

In [27]:
# Save map
outfp = 'data/leaflet_folium_basemap.html'
m.save(outfp)

In [28]:
from IPython.display import HTML, display
HTML('<iframe src='+outfp+' width=700 height=400> </iframe>')

In [29]:
# Adding some layers to the map

# Filepaths
fp = 'data/Vaestotietoruudukko_2015.shp'
add_fp = 'data/addresses.shp'

# Read data
data = gpd.read_file(fp)
a_data = gpd.read_file(add_fp)


In [30]:
# Re-project coordinates
data['geometry'] = data['geometry'].to_crs(epsg=4326)
a_data['geometry'] = a_data['geometry'].to_crs(epsg=4326)

In [31]:
# Update gdf CRS
data.crs = fiona.crs.from_epsg(4326)
a_data.crs = fiona.crs.from_epsg(4326)
print(data.crs, '\t', a_data.crs)

{'init': 'epsg:4326', 'no_defs': True} 	 {'init': 'epsg:4326', 'no_defs': True}


In [32]:
# Select data
data_select = data.copy().loc[(data['ASUKKAITA'] > 0) & (data['ASUKKAITA'] < 1000)]

In [33]:
# Create a Geo-id which is needed by the Folium 
# (it needs to have a unique identifier for each row)
data_select['geoid'] = data_select.index.astype(str)
a_data['geoid'] = a_data.index.astype(str)

In [34]:
# Select map data
map_data = data_select.copy()[['geoid', 'ASUKKAITA', 'geometry']]

In [35]:
# Save file as GeoJSON
map_json = map_data.to_json()

In [40]:
## Map creation

# Create a map instance
m = folium.Map(
    location=[60.25, 24.8],
    tiles='Stamen Terrain',
    zoom_start=11,
    control_scale=True,
    prefer_canvas=True #Optional
)


# Create a Clustered map where points are clustered
marker_cluster = MarkerCluster().add_to(m)

In [41]:
# Create Choropleth map where the colors are coming from a column "ASUKKAITA".
# Notice: 'geoid' column that we created earlier needs to be assigned always as the first column
# with threshold_scale we can adjust the class intervals for the values
m.choropleth(geo_data=map_json, data=map_data, columns=['geoid', 'ASUKKAITA'], key_on="feature.id",
             fill_color='YlOrRd', fill_opacity=0.7, line_opacity=0.2,
             line_color='white', line_weight=0,
             threshold_scale=[100, 250, 500, 1000, 2000], 
             legend_name='Population in Helsinki', highlight=False, smooth_factor=1.0)

In [42]:
# Create Address points on top of the map
for idx, row in a_data.iterrows():
    # Get lat and lon of points
    lon = row['geometry'].x
    lat = row['geometry'].y

    # Get address information
    address = row['address']
    
    # Add marker to the map
    folium.Marker(location=[lat, lon], 
                  popup=address).add_to(marker_cluster)

In [43]:
# Save output
leaflet_map = 'output/leaflet_pop15.html'
m.save(leaflet_map)

HTML('<iframe src='+leaflet_map+' width=700 height=400> </iframe>')